In [1]:
import pandas as pd
import os
import numpy as np
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from pylab import *

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42


In [21]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
path = 'cdf_result/'
file_list = os.listdir(path)

file_list = []
balanceset = [
            'veh', 
              'opt',
              'pen',
              'letter' ,
    
    
 
            'glass',
            'ecoli',
            'yeast',
            'satimage',

            'page',

             ]
for f in os.listdir(path):
    if 'xshift' in f and 'optopt' not in f and 'soften' not in f:
        for k in balanceset:
            if k in f and 'unknown' not in f:
                file_list.append(f)

In [22]:
file_list

['yeastxshift.csv',
 'satimagexshift.csv',
 'optxshift.csv',
 'vehxshift.csv',
 'glassxshift.csv',
 'penxshift.csv',
 'letterxshift.csv',
 'pagexshift.csv',
 'ecolixshift.csv']

In [24]:
dataframe = []

for f in file_list:
    data = pd.read_csv('cdf_result/' + f)
    
    col = data.columns[1:]
    data = data[col]
    
    
    dmrobust = list(data[col].iloc[4,:][1:])
    dmdr = list(data[col].iloc[5,:][1:])
    sndmdr = list(data[col].iloc[6,:][1:])
    min_dmrobust = ['min_dmrobust'] + list(np.min(np.concatenate((dmrobust, dmdr, sndmdr),axis = 0).reshape(3,len(sndmdr)),axis = 0))
    data.loc[len(data)] = min_dmrobust
    
    robust = list(data[col].iloc[7,:][1:])
    triple = list(data[col].iloc[8,:][1:])
    sntriple = list(data[col].iloc[9,:][1:])
    min_robust = ['min_robust'] + list(np.min(np.concatenate((robust, triple, sntriple),axis = 0).reshape(3,len(robust)),axis = 0))
    data.loc[len(data)] = min_robust

    
    robust = list(data[col].iloc[10,:][1:])
    triple = list(data[col].iloc[11,:][1:])
    sntriple = list(data[col].iloc[12,:][1:])
    min_robust = ['min_robustr'] + list(np.min(np.concatenate((robust, triple, sntriple),axis = 0).reshape(3,len(robust)),axis = 0))
    data.loc[len(data)] = min_robust
    dataframe.append(data)
    
    data.to_csv('final/' + f + 'xshiftknown.csv',index = False)
    
    

In [25]:

logging_policy_list = ['[0.95, 0.1, 0]', '[0.7, 0.1, 0]','[0.99]','[0.1]','[1]','[0.91]','[0.95]']
target_policy = [1]
xshift_logging = ['[15, 1, 0]','[12, 1, 0]','[9, 1, 0]','[6, 1, 0]','[4, 1, 0]','[2, 1, 0]',\
                  '[1.5, 2, 1]','[0.6, 2, 1]','[1.5, 3, 1]','[2, 2, 1]']

todolist = logging_policy_list + xshift_logging

dataframe_todolist = pd.DataFrame(columns = ['dmrobust','robust','robust-r', \
                                             'sndmdr','sntriple','sntriple-r',\
                                            'snips','snips-r','dmrobust-f','robust-f','robust-r-f'])



gaussian_xshift_logging = ['[15, 1, 0]','[12, 1, 0]','[9, 1, 0]','[6, 1, 0]','[4, 1, 0]','[2, 1, 0]']
dic_robust = {}
dic_robust_r = {}
for s in gaussian_xshift_logging:
    dic_robust_r[s] = []
    dic_robust[s] = []

for t in todolist:
    
    dmrobust_c = 0
    robust_c = 0
    robust_r_c = 0
    
    sndmdr_c = 0
    sntriple_c = 0
    sntriple_r_c = 0
    
    dmrobust_f_c = 0
    robust_f_c = 0
    robust_r_f_c = 0
    snips_f_c = 0
    snips_r_f_c = 0
    
    
    for _,data in enumerate(dataframe):
        for c in col[1:]:
            if t in c:
                dmrobust = data[c][4]
                robust = data[c][7]
                robust_r = data[c][10]
                if t in gaussian_xshift_logging:
                    dic_robust[t].append(robust)
                    dic_robust_r[t].append(robust_r)

#                 snipsr = data[c][3]
#                 snips = data[c][1]
                miv = min(dmrobust,robust,robust_r)
                if dmrobust == miv:
                    dmrobust_c += 1
                elif robust == miv:
                    robust_c += 1
                elif robust_r == miv:
                    robust_r_c += 1
                
#               sn
                sndmdr = data[c][6]
                sntriple = data[c][9]
                sntriple_r = data[c][12]
            
                miv = min(sndmdr,sntriple,sntriple_r)
                if sndmdr == miv:
                    sndmdr_c += 1
                elif sntriple == miv:
                    sntriple_c += 1
                elif sntriple_r == miv:
                    sntriple_r_c += 1
                
                dmrobust_f = data[c][15]
                robust_f = data[c][16]
                robust_r_f = data[c][17]
                snipsr = data[c][3]
                snips = data[c][1]
                miv = min(dmrobust_f,robust_f,robust_r_f,snipsr, snips )
                if dmrobust_f == miv:
                    dmrobust_f_c += 1
                elif robust_f == miv:
                    robust_f_c += 1
                elif robust_r_f == miv:
                    robust_r_f_c += 1
                elif snipsr == miv:
                    snips_r_f_c += 1
                else:
                    snips_f_c += 1
                    

    print(t)
    dataframe_todolist.loc[len(dataframe_todolist)] = [dmrobust_c,robust_c,robust_r_c,\
                                                     sndmdr_c, sntriple_c,sntriple_r_c,\
                                                      snips_f_c,snips_r_f_c, dmrobust_f_c,robust_f_c,robust_r_f_c]

    
dataframe_todolist['method'] = todolist
dataframe_todolist


[0.95, 0.1, 0]
[0.7, 0.1, 0]
[0.99]
[0.1]
[1]
[0.91]
[0.95]
[15, 1, 0]
[12, 1, 0]
[9, 1, 0]
[6, 1, 0]
[4, 1, 0]
[2, 1, 0]
[1.5, 2, 1]
[0.6, 2, 1]
[1.5, 3, 1]
[2, 2, 1]


dmrobust  robust  robust-r  sndmdr  sntriple  sntriple-r  snips  snips-r  \
0         37      40       103      26        36         118     16        3   
1         36      31       113       6        48         126     21       17   
2         11      48       121      25        72          83      1        1   
3         20      54       106      60        73          47      8        5   
4         14      22       144      24        81          75      4        3   
5          7      24       149       8        96          76      5        1   
6          8      26       146      10        89          81      6        4   
7          8      19        99      15        33          78      3        4   
8         13      21        92       7        46          73      4        7   
9         14      22        90      14        33          79      0        2   
10        11      20        95      16        32          78      5        6   
11        13      30        83      15        31          80      3        3   
12        13      32        81      19        37          70      1        6   
13        16      29        81      18        66          42      9        1   
14        12      26        88      15        74          37     13        0   
15        18      21        87      18        72          36     10        2   
16        15      25        86      22        71          33     13        3   

    dmrobust-f  robust-f  robust-r-f          method  
0           28        31         102  [0.95, 0.1, 0]  
1            9        26         107   [0.7, 0.1, 0]  
2           11        76          91          [0.99]  
3           23        72          72           [0.1]  
4           19        57          97             [1]  
5            5        60         109          [0.91]  
6            5        55         110          [0.95]  
7            8        23          88      [15, 1, 0]  
8            8        33          74      [12, 1, 0]  
9            9        34          81       [9, 1, 0]  
10          12        29          74       [6, 1, 0]  
11           8        34          78       [4, 1, 0]  
12          13        33          73       [2, 1, 0]  
13          14        48          54     [1.5, 2, 1]  
14           9        46          58     [0.6, 2, 1]  
15           9        45          60     [1.5, 3, 1]  
16          10        52          48       [2, 2, 1]

In [26]:
# IPS


[                     model  [0.95, 0.1, 0]_1_[15, 1, 0]  \
 0                known ips                      0.09077   
 1              known snips                      0.09151   
 2              known ips-r                      0.03014   
 3            known snips-r                      0.02336   
 4           known dmrobust                      0.04775   
 5               known dmdr                      0.08250   
 6             known sndmdr                      0.08264   
 7             known robust                      0.03295   
 8             known triple                      0.05359   
 9           known sntriple                      0.05364   
 10          known robust-r                      0.02448   
 11          known triple-r                      0.02131   
 12        known sntriple-r                      0.02265   
 13    known triple-robustr                      0.06914   
 14  known sntriple-robustr                      0.06938   
 15            min_dmrobust             

In [27]:

logging_policy_list = ['[0.95, 0.1, 0]', '[0.7, 0.1, 0]','[0.99]','[0.1]','[1]','[0.91]','[0.95]']
target_policy = [1]
xshift_logging = ['[15, 1, 0]','[12, 1, 0]','[9, 1, 0]','[6, 1, 0]','[4, 1, 0]','[2, 1, 0]',\
                  '[1.5, 2, 1]','[0.6, 2, 1]','[1.5, 3, 1]','[2, 2, 1]']

todolist = logging_policy_list + xshift_logging

dataframe_todolist = pd.DataFrame(columns = ['robust-r', \
                                             'sntriple-r',\
                                            ])



gaussian_xshift_logging = ['[15, 1, 0]','[12, 1, 0]','[9, 1, 0]','[6, 1, 0]','[4, 1, 0]','[2, 1, 0]']
dic_robust = {}
dic_robust_r = {}
for s in gaussian_xshift_logging:
    dic_robust_r[s] = []
    dic_robust[s] = []

for t in todolist:
    
    dmrobust_c = 0
    robust_c = 0
    robust_r_c = 0
    
    sndmdr_c = 0
    sntriple_c = 0
    sntriple_r_c = 0
    
    dmrobust_f_c = 0
    robust_f_c = 0
    robust_r_f_c = 0
    snips_f_c = 0
    snips_r_f_c = 0
    
    
    for _,data in enumerate(dataframe):
        for c in col[1:]:
            if t in c:
                robust_r = data[c][10]
                sntriple_r = data[c][12]
                miv = min(robust_r,sntriple_r)
                
                if robust_r == miv:
                    robust_r_c += 1
                elif sntriple_r == miv:
                    sntriple_r_c += 1
                
                
                    

    print(t)
    dataframe_todolist.loc[len(dataframe_todolist)] = [robust_r_c,sntriple_r_c]

    
dataframe_todolist['method'] = todolist
dataframe_todolist


[0.95, 0.1, 0]
[0.7, 0.1, 0]
[0.99]
[0.1]
[1]
[0.91]
[0.95]
[15, 1, 0]
[12, 1, 0]
[9, 1, 0]
[6, 1, 0]
[4, 1, 0]
[2, 1, 0]
[1.5, 2, 1]
[0.6, 2, 1]
[1.5, 3, 1]
[2, 2, 1]


robust-r  sntriple-r          method
0         69         111  [0.95, 0.1, 0]
1         60         120   [0.7, 0.1, 0]
2        129          51          [0.99]
3        114          66           [0.1]
4         63         117             [1]
5         74         106          [0.91]
6         86          94          [0.95]
7         59          67      [15, 1, 0]
8         61          65      [12, 1, 0]
9         55          71       [9, 1, 0]
10        58          68       [6, 1, 0]
11        53          73       [4, 1, 0]
12        48          78       [2, 1, 0]
13        61          65     [1.5, 2, 1]
14        66          60     [0.6, 2, 1]
15        66          60     [1.5, 3, 1]
16        68          58       [2, 2, 1]

In [28]:
dataframe[0]

model  [0.95, 0.1, 0]_1_[15, 1, 0]  \
0                known ips                      0.09077   
1              known snips                      0.09151   
2              known ips-r                      0.03014   
3            known snips-r                      0.02336   
4           known dmrobust                      0.04775   
5               known dmdr                      0.08250   
6             known sndmdr                      0.08264   
7             known robust                      0.03295   
8             known triple                      0.05359   
9           known sntriple                      0.05364   
10          known robust-r                      0.02448   
11          known triple-r                      0.02131   
12        known sntriple-r                      0.02265   
13    known triple-robustr                      0.06914   
14  known sntriple-robustr                      0.06938   
15            min_dmrobust                      0.04775   
16              min_robust                      0.03295   
17             min_robustr                      0.02131   

    [0.95, 0.1, 0]_1_[12, 1, 0]  [0.95, 0.1, 0]_1_[9, 1, 0]  \
0                       0.06804                     0.05622   
1                       0.06824                     0.05487   
2                       0.02751                     0.01905   
3                       0.02713                     0.02160   
4                       0.05840                     0.04250   
5                       0.08157                     0.03357   
6                       0.07806                     0.03373   
7                       0.04080                     0.03871   
8                       0.04757                     0.03708   
9                       0.04765                     0.03629   
10                      0.03360                     0.03233   
11                      0.02455                     0.01895   
12                      0.02399                     0.01998   
13                      0.04073                     0.04209   
14                      0.04101                     0.04115   
15                      0.05840                     0.03357   
16                      0.04080                     0.03629   
17                      0.02399                     0.01895   

    [0.95, 0.1, 0]_1_[6, 1, 0]  [0.95, 0.1, 0]_1_[4, 1, 0]  \
0                      0.03472                     0.02516   
1                      0.04146                     0.02560   
2                      0.02183                     0.02168   
3                      0.02356                     0.02453   
4                      0.02928                     0.04154   
5                      0.05088                     0.02350   
6                      0.05115                     0.02349   
7                      0.03821                     0.05541   
8                      0.02625                     0.01993   
9                      0.02638                     0.01983   
10                     0.02522                     0.04638   
11                     0.01499                     0.02518   
12                     0.01456                     0.02377   
13                     0.02403                     0.03095   
14                     0.02300                     0.03082   
15                     0.02928                     0.02349   
16                     0.02625                     0.01983   
17                     0.01456                     0.02377   

    [0.95, 0.1, 0]_1_[2, 1, 0]  [0.95, 0.1, 0]_1_[1.5, 3, 1]  \
0                      0.02275                       0.03778   
1                      0.02387                       0.03617   
2                      0.02004                       0.02104   
3                      0.02183                       0.01867   
4                      0.05622                       0.04700   
5                      0.01803                       0.02245   
6                      0.01874                       0.02278   
7         

In [32]:

logging_policy_list = ['[0.95, 0.1, 0]', '[0.7, 0.1, 0]','[0.99]','[0.1]','[1]','[0.91]','[0.95]']
target_policy = [1]
xshift_logging = ['[15, 1, 0]','[12, 1, 0]','[9, 1, 0]','[6, 1, 0]','[4, 1, 0]','[2, 1, 0]',\
                  '[1.5, 2, 1]','[0.6, 2, 1]','[1.5, 3, 1]','[2, 2, 1]']

todolist = logging_policy_list + xshift_logging

dataframe_todolist = pd.DataFrame(columns = ['robust-r', \
                                             'sntriple-r',\
                                             'ips_sntriple'
                                            ])



gaussian_xshift_logging = ['[15, 1, 0]','[12, 1, 0]','[9, 1, 0]','[6, 1, 0]','[4, 1, 0]','[2, 1, 0]']
dic_robust = {}
dic_robust_r = {}
for s in gaussian_xshift_logging:
    dic_robust_r[s] = []
    dic_robust[s] = []

for t in todolist:
    
    dmrobust_c = 0
    robust_c = 0
    robust_r_c = 0
    
    sndmdr_c = 0
    sntriple_c = 0
    sntriple_r_c = 0
    
    dmrobust_f_c = 0
    robust_f_c = 0
    robust_r_f_c = 0
    snips_f_c = 0
    snips_r_f_c = 0
    
    ips_sntriple_r_c = 0
    
    
    for _,data in enumerate(dataframe):
        for c in col[1:]:
            if t in c:
                robust_r = data[c][10]
                sntriple_r = data[c][12]
                ips_sntriple_r = data[c][14]
                robust_r = 1000
                miv = min(robust_r,sntriple_r,ips_sntriple_r)
                
                if robust_r == miv:
                    robust_r_c += 1
                elif sntriple_r == miv:
                    sntriple_r_c += 1
                else:
                    ips_sntriple_r_c +=1
                
                
                    

    print(t)
    dataframe_todolist.loc[len(dataframe_todolist)] = [robust_r_c,sntriple_r_c, ips_sntriple_r_c]

    
dataframe_todolist['method'] = todolist
dataframe_todolist


[0.95, 0.1, 0]
[0.7, 0.1, 0]
[0.99]
[0.1]
[1]
[0.91]
[0.95]
[15, 1, 0]
[12, 1, 0]
[9, 1, 0]
[6, 1, 0]
[4, 1, 0]
[2, 1, 0]
[1.5, 2, 1]
[0.6, 2, 1]
[1.5, 3, 1]
[2, 2, 1]


robust-r  sntriple-r  ips_sntriple          method
0          0         121            59  [0.95, 0.1, 0]
1          0         102            78   [0.7, 0.1, 0]
2          0          77           103          [0.99]
3          0          83            97           [0.1]
4          0          97            83             [1]
5          0          83            97          [0.91]
6          0          82            98          [0.95]
7          0          66            60      [15, 1, 0]
8          0          57            69      [12, 1, 0]
9          0          65            61       [9, 1, 0]
10         0          66            60       [6, 1, 0]
11         0          73            53       [4, 1, 0]
12         0          74            52       [2, 1, 0]
13         0          62            64     [1.5, 2, 1]
14         0          65            61     [0.6, 2, 1]
15         0          59            67     [1.5, 3, 1]
16         0          58            68       [2, 2, 1]

In [ ]:

logging_policy_list = ['[0.95, 0.1, 0]', '[0.7, 0.1, 0]','[0.99]','[0.1]','[1]','[0.91]','[0.95]']
target_policy = [1]
xshift_logging = ['[15, 1, 0]','[12, 1, 0]','[9, 1, 0]','[6, 1, 0]','[4, 1, 0]','[2, 1, 0]',\
                  '[1.5, 2, 1]','[0.6, 2, 1]','[1.5, 3, 1]','[2, 2, 1]']

todolist = logging_policy_list + xshift_logging

dataframe_todolist = pd.DataFrame(columns = ['robust-r', \
                                             'sntriple-r',\
                                             'ips_sntriple'
                                            ])



gaussian_xshift_logging = ['[15, 1, 0]','[12, 1, 0]','[9, 1, 0]','[6, 1, 0]','[4, 1, 0]','[2, 1, 0]']
dic_robust = {}
dic_robust_r = {}
for s in gaussian_xshift_logging:
    dic_robust_r[s] = []
    dic_robust[s] = []

for t in todolist:
    
    dmrobust_c = 0
    robust_c = 0
    robust_r_c = 0
    
    sndmdr_c = 0
    sntriple_c = 0
    sntriple_r_c = 0
    
    dmrobust_f_c = 0
    robust_f_c = 0
    robust_r_f_c = 0
    snips_f_c = 0
    snips_r_f_c = 0
    
    ips_sntriple_r_c = 0
    
    
    for _,data in enumerate(dataframe):
        for c in col[1:]:
            if t in c:
                robust_r = data[c][10]
                sntriple_r = data[c][12]
                ips_sntriple_r = data[c][14]
                robust_r = 1000
                miv = min(robust_r,sntriple_r,ips_sntriple_r)
                
                if robust_r == miv:
                    robust_r_c += 1
                elif sntriple_r == miv:
                    sntriple_r_c += 1
                else:
                    ips_sntriple_r_c +=1
                
                
                    

    print(t)
    dataframe_todolist.loc[len(dataframe_todolist)] = [robust_r_c,sntriple_r_c, ips_sntriple_r_c]

    
dataframe_todolist['method'] = todolist
dataframe_todolist
